In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

from tqdm.notebook import tqdm

In [2]:
db = pd.read_parquet(
    "/home/sergey/projects/Credit_scoring/raw/train_data/train_data_0.pq", 
    engine="pyarrow"
)

In [12]:
db.shape

(1974724, 61)

In [4]:
db.describe().iloc[1:, 1:]

,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,...,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
mean,6.537471,9.271325,8.330264,8.299071,8.309444,6.991781,8.337011,9.740888,2.399848,3.012951,...,2.313242,2.343014,2.376717,3.576724,1.027562,2.694898,3.353936,1.002386,0.172138,0.223934
std,5.121167,5.773757,4.593511,5.245598,4.464844,5.184165,4.233992,5.799625,1.376391,0.683844,...,1.247674,1.228473,1.205555,1.035586,0.276702,0.489136,1.057304,0.059093,0.377501,0.416878
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,4.000000,5.000000,4.000000,6.000000,1.000000,5.000000,5.000000,2.000000,3.000000,...,3.000000,3.000000,3.000000,4.000000,1.000000,2.000000,3.000000,1.000000,0.000000,0.000000
50%,5.000000,9.000000,9.000000,8.000000,8.000000,6.000000,10.000000,10.000000,2.000000,3.000000,...,3.000000,3.000000,3.000000,4.000000,1.000000,3.000000,4.000000,1.000000,0.000000,0.000000
75%,9.000000,14.000000,11.000000,13.000000,12.000000,12.000000,11.000000,15.000000,2.000000,3.000000,...,3.000000,3.000000,3.000000,4.000000,1.000000,3.000000,4.000000,1.000000,0.000000,0.000000
max,51.000000,19.000000,17.000000,17.000000,16.000000,16.000000,15.000000,19.000000,6.000000,5.000000,...,3.000000,3.000000,3.000000,4.000000,6.000000,6.000000,5.000000,3.000000,1.000000,1.000000


In [7]:
corr_matrix = db.corr(method='spearman').abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

# Drop features 
df = db.drop(to_drop, axis=1)

/tmp/ipykernel_4574/1128466963.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))


In [11]:
df.shape

(1974724, 51)

In [4]:
round(db.describe().iloc[1, 1] + db.describe().iloc[2, 1] * 3)

22

In [58]:
arr = db.values[0].astype(np.int8)
arr

array([ 0,  1, 18,  9,  2,  3, 16, 10, 11,  3,  3,  0,  2, 11,  6, 16,  5,
        4,  8,  1,  1,  1,  1,  1, 16,  2, 17,  1,  1,  1,  0,  0,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  4,  3,  3,  3,  3,  3,  3,  3,  3,  4,
        3,  3,  3,  4,  1,  3,  4,  1,  0,  0], dtype=int8)

In [59]:
obj = arr.tobytes()

In [60]:
obj

b'\x00\x01\x12\t\x02\x03\x10\n\x0b\x03\x03\x00\x02\x0b\x06\x10\x05\x04\x08\x01\x01\x01\x01\x01\x10\x02\x11\x01\x01\x01\x00\x00\x03\x03\x03\x03\x03\x03\x03\x03\x03\x04\x03\x03\x03\x03\x03\x03\x03\x03\x04\x03\x03\x03\x04\x01\x03\x04\x01\x00\x00'

In [63]:
np.frombuffer(obj, dtype='int8')

array([ 0,  1, 18,  9,  2,  3, 16, 10, 11,  3,  3,  0,  2, 11,  6, 16,  5,
        4,  8,  1,  1,  1,  1,  1, 16,  2, 17,  1,  1,  1,  0,  0,  3,  3,
        3,  3,  3,  3,  3,  3,  3,  4,  3,  3,  3,  3,  3,  3,  3,  3,  4,
        3,  3,  3,  4,  1,  3,  4,  1,  0,  0], dtype=int8)